# Main Commands

## Fetch Images

### Download card images from Scryfall queries

In [ ]:
from art_scan import process_query

queries = [
]

force_scryfall = [
]

force_mtgpics = [
]

[process_query(q) for q in queries]
[process_query(q, skip_mtgpics=True) for q in force_scryfall]
[process_query(q, skip_scryfall=True) for q in force_mtgpics]

## Fetch Set Symbols

### Parse Keyrune website cheatsheet and print out UTF8 representation

In [ ]:
from bs4 import BeautifulSoup
import requests

url = "https://keyrune.andrewgioia.com/cheatsheet.html"

response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
set_symbol_spans = soup.findAll("span", {"class": "utf"})
print(f"Found {len(set_symbol_spans)} icons.")

In [ ]:
set_symbols = {}
for set_symbol_span in set_symbol_spans:
    set_symbol = str(set_symbol_span.findChild("i").get_text())
    try:
        set_code = set_symbol_span.get_text().split("ss-")[1].split(" ")[0].upper()
    except IndexError as e:
        continue
    finally:
        set_symbols[set_code] = set_symbol

for set_code, set_symbol in set_symbols.items():
    print(f"\"{set_code}\": \"{set_symbol}\",")

## Inventory Management

### Check for image files

In [ ]:
from lib.helpers import os_helpers

ART_FOLDER = "C:/Users/evanh/My Drive/Hobbies/MTG/Proxies/art"

to_find = [
]

missing_cards = os_helpers.find_missing_files(ART_FOLDER, to_find)
missing_cards_str = "\n".join([f"\t\"{card}\"," for card in missing_cards])
print(f"Missing:\n[\n{missing_cards_str}\n]")

### Generate inventory as TSV

#### Load proxy image files as `InventoryCard`s

In [3]:
from lib.helpers import os_helpers, inventory_helpers

PROXY_FOLDER = "C:/Users/evanh/My Drive/Hobbies/MTG/Proxies/mpc/all"
TOKEN_FOLDER = "C:/Users/evanh/My Drive/Hobbies/MTG/Proxies/mpc/tokens"
CUSTOM_FOLDER = "C:/Users/evanh/My Drive/Hobbies/MTG/Proxies/mpc/custom"
IGNORED_FILES = ["desktop.ini", "inventory.tsv", "fix", "done", "Josh's Bachelor Party", "LOTR"]

failed = []
inventory_cards = []

# Import "normal" cards
for file_name in os_helpers.list_files(PROXY_FOLDER, ignore=IGNORED_FILES):
    try:
        inventory_cards.append(inventory_helpers.create_normal_inventory_card(file_name, PROXY_FOLDER))
    except Exception as e:
        print(f"FAILURE: {e}")
        failed.append(file_name)

# Import tokens
for file_name in os_helpers.list_files(TOKEN_FOLDER, ignore=IGNORED_FILES):
    try:
        inventory_cards.append(inventory_helpers.create_token_inventory_card(file_name, TOKEN_FOLDER))
    except Exception as e:
        print(f"FAILURE: {e}")
        failed.append(file_name)

# Import custom cards
for file_name in os_helpers.list_files(CUSTOM_FOLDER, ignore=IGNORED_FILES):
    try:
        inventory_cards.append(inventory_helpers.create_custom_inventory_card(file_name, CUSTOM_FOLDER))
    except Exception as e:
        print(f"FAILURE: {e}")
        failed.append(file_name)

if len(failed) > 0:
    print("The following files were unable to be imported:")
    print(failed)

[str(c) for c in inventory_cards]

Searching Scryfall: "Anara, Wolvid Familiar"... Found: Anara, Wolvid Familiar (Jesper Ejsing) [CMR]
Searching Scryfall: "Chevill, Bane of Monsters"... Found: Chevill, Bane of Monsters (Yongjae Choi) [IKO]
Searching Scryfall: "Explore"... Found: Explore (John Avon) [OTC]
Searching Scryfall: "Forest"... Found: Forest (Alayna Danner) [BLB]
Searching Scryfall: "Mountain"... Found: Mountain (Samuele Bandini) [BLB]
Searching Scryfall: "Rin and Seri, Inseparable"... Found: Rin and Seri, Inseparable (Leesha Hannigan) [M21]
Searching Scryfall: "Yoshimaru, Ever Faithful"... Found: Yoshimaru, Ever Faithful (Ilse Gort) [NEC]


['Anara, Wolvid Familiar (Fenrir, the Mighty),PRX,Custom,Fullart,Creature,Green,2023-03-06T22:31:22Z,',
 'Chevill, Bane of Monsters (Boba Fett, Bounty Hunter),PRX,Custom,Fullart,Creature,Multi,2024-08-27T15:39:08Z,',
 'Explore,PRX,Custom,Extended,Sorcery,Green,2024-08-27T14:51:43Z,',
 'Forest (Dog),PRX,Dall-E,Fullart,Land,Green,2023-03-06T22:05:58Z,',
 'Mountain (Dog),PRX,Dall-E,Fullart,Land,Red,2023-03-06T22:13:37Z,',
 'Rin and Seri, Inseparable (Kevin and Clark, Inseparable),PRX,Custom,Fullart,Creature,Multi,2023-03-06T22:35:00Z,',
 'Yoshimaru, Ever Faithful (Ross, Never Forgotten),PRX,Custom,Fullart,Creature,White,2023-03-06T22:32:06Z,']

#### Sort and write cards to `.tsv` format

In [ ]:
import operator

INVENTORY_FILE = "C:/Users/evanh/My Drive/Hobbies/MTG/Proxies/mpc/inventory.tsv"

# Sort inventory to match destination spreadsheet
sorted_cards = sorted(inventory_cards, key=operator.itemgetter("type_", "name"))

# Write list of cards to TSV
failed = []
with open(INVENTORY_FILE, "w") as f:
    for card in sorted_cards:
        try:
            f.write(card.tsv + "\n")
        except UnicodeEncodeError as e:
            print("Unable to write card due to unsupported character.")
            print(e)
            failed.append(card)
             
    print(f"Created inventory at: \"{INVENTORY_FILE}\"")

if len(failed) > 0:
    print("The following cards were unable to be written:")
    print([f.name for f in failed])

### Load orders into inventory

In [ ]:
import json5 as json
from lib.classes import OrderCard

orders = {
    "Evan": "orders/evan.json",
    "Blake": "orders/blake.json",
    "Matt": "orders/matt.json",
    "Scott": "orders/scott.json",
}

failed = []
total = 0
for name, list_file in orders.items():
    with open(list_file, "r") as f:
        order_list = json.load(f)

        # Import decklist into a list of `OrderCard`s
        try:
            order_cards = OrderCard.import_list(order_list, name)
        except ValueError as e:
            print(f"Unable to import {list_file}.")
            order_cards = []

        # Add ordered cards to inventory counts
        for card in order_cards:
            try:
                inventory.add_to_order(card)
                total += card.count
            except ValueError as e:
                print(e)
                print(f"Unable to add card: {card.name}")
                failed.append(card)

if len(failed) > 0:
    failed_str = "\n\t".join([f"{c.name}, {c.set_code.upper()}" for c in failed])
    print(f"Failed cards:\n\t{failed_str}")

inventory_cards = inventory.cards  # To use in cell above to write out to TSV

In [ ]:
total

## Order Generation

### Pull inventory of proxies from Google Sheets (public)

In [ ]:
from lib import common
from lib.classes import Inventory

sheet_id = '1JbJy3kM34XUOPud4qPaAhp2luU0Qgidq4CPQVm0TTw0'
url = f'https://docs.google.com/spreadsheet/ccc?key={sheet_id}&output=csv'

proxy_csv_data = common.load_csv(url.format(sheet_id))
inventory = Inventory.from_csv(proxy_csv_data)
print(inventory)

### Pull inventory of proxies from  Google Sheets (private)

In [ ]:
from googleapiclient import discovery
from lib.google_auth import get_creds

args = dict(
    spreadsheetId='13ofTLZFmPMkT09Np2US8KF_JmTfgqdeQ5aeY16L43s0',
    range='Inventory!A:L',
)

service = discovery.build('sheets', 'v4', credentials=get_creds())
result = service.spreadsheets().values().get(**args).execute()
proxy_csv_data = list(result.get('values', []))
inventory = Inventory.from_csv(proxy_csv_data)
print(inventory)

### Write out files for order

In [ ]:
proxies = [p for p in proxies_to_print if not p.is_mdfc]
# proxies = [p for p in proxies_to_print if p.is_mdfc]
len(proxies)

In [ ]:
# TODO: Fix this now that proxies are not in subfolders

from lib.helpers import inventory_helpers, os_helpers

folder_size = 612
prefix_name = "proxies_{number:02d}"
landing_folder = "C:/Users/evanh/Temp/landing"
output_folder = "C:/Users/evanh/Temp/mpc"
proxy_folder = "C:/Users/evanh/My Drive/Hobbies/MTG/Proxies/mpc-ready"

# Write all files out to landing zone
missing, duplicates = inventory_helpers.create_unique_proxies(proxies, proxy_folder, landing_folder)
if len(missing):
    print("Missing:\n\t{}".format('\n\t'.join([p.name for p in missing])))
if len(duplicates):
    print("Duplicates:\n\t{}".format('\n\t'.join([p.name for p in duplicates])))

# Move files into subfolders of fixed size (`folder_size`)
output_folders = os_helpers.split_files(landing_folder, output_folder, folder_size, prefix_name)
output_folders

### Move proxy files

In [ ]:
import os
from lib.helpers import os_helpers

art_folder = "C:/Users/evanh/My Drive/Hobbies/Hobbies/MTG/Proxies/mpc/art"
proxy_folder = "C:/Users/evanh/My Drive/Hobbies/MTG/Hobbies/Proxies/mpc-ready"
destination_folder = "C:/Users/evanh/My Drive/MTG/Hobbies/Proxies/mpc/all"

ignored_files = [
    "desktop.ini",
    "Custom",
    "inventory.tsv",
]

# Iterate through art files to hold details in memory
art = {}
for file_name in os_helpers.list_files(art_folder, ignore=ignored_files):
    try:
        name = file_name[:file_name.index("(")-1]
        artist = file_name[file_name.index("(")+1:file_name.index(")")]
        set_code = file_name[file_name.index("[")+1:file_name.index("]")]
    except ValueError as e:
        print(f"Unable to parse {file_name}")

    # Store results in a list to handle cards that have multiple versions
    if name not in art:
        art[name] = [(artist, set_code)]
    else:
        art[name].append((artist, set_code))

# Iterate through proxy files to rename & move them based on art files
for typed_proxy_folder in os_helpers.list_files(proxy_folder, ignore=ignored_files, include_base=True):
    # for original_proxy_file in os_helpers.list_files(typed_proxy_folder, ignore=ignored_files)[:3]:
    for original_proxy_file in os_helpers.list_files(typed_proxy_folder, ignored_files):
        # Extract attributes from proxy (if available)
        file_extension = original_proxy_file.split(".")[-1]
        proxy_file = "".join(".".join(original_proxy_file.split(".")[:-1]))  # Trim file extension

        try:
            proxy_artist = proxy_file[proxy_file.index("[")+1:proxy_file.index("]")]
            proxy_file = proxy_file.replace(f" [{proxy_artist}]", "")
        except ValueError:
            proxy_artist = ""

        try:
            frame_type = proxy_file[proxy_file.index("(")+1:proxy_file.index(")")]
            proxy_file = proxy_file.replace(f" ({frame_type})", "")
        except ValueError:
            frame_type = ""

        name = proxy_file  # If original file name had extras, try-except blocks would remove them

        # Look up art
        try:
            versions = art[name]
        except KeyError:
            print(f"Art not found for: {name}")
            continue
        
        # Process one to many tuples of (artist, set_code) associated with card name
        if len(versions) == 1:
            artist, set_code = versions[0]
            new_file_name = f"{name} ({artist}, {set_code}){' [' + frame_type + ']' if frame_type else ''}.{file_extension}"
            os.rename(f"{typed_proxy_folder}/{original_proxy_file}", f"{destination_folder}/{new_file_name}")
        else:
            print(f"Multiple versions found: {versions}")
            continue

# Debug Commands

## Save card info to `card.json`

In [ ]:
from scripts import get_card_info

args = "\"Angelic Accord\" \"M14\""
get_card_info.main(args)

## Scryfall

### Get multiple cards

In [ ]:
from lib.helpers import scryfall_helpers

queries = [
    "Crucible of Worlds",
]

for q in queries:
    cards = scryfall_helpers.get_matched_cards(q)
    [print(f"\t{repr(c)}") for c in cards]

### Get single card

In [ ]:
from lib.helpers import scryfall_helpers

queries = [
    ("Crucible of Worlds", "5DN")
]

for name, set_code in queries:
    card = scryfall_helpers.get_named_card(name, set_code)
    print(repr(card))

## MTGPICS.com

### Find `gamerid`

In [ ]:
from lib.helpers import mtgpics_helpers

queries = [
    "Crucible of Worlds",
]

for q in queries:
    card_name, gamerid = mtgpics_helpers.get_gamerid(q)
    print(f"Found id [{gamerid}] for [{card_name}]")

### Find image versions by `gamerid`

In [ ]:
from lib.helpers import mtgpics_helpers

to_find = "Crucible of Worlds"
card_name, gamerid = to_find, mtgpics_helpers.get_gamerid(to_find)
versions = mtgpics_helpers.find_all_art_versions(card_name, gamerid)
versions

## Misc

In [ ]:
all([v in "abc" for v in ["a", "x"]])

In [23]:
!pip freeze > requirements.txt